In [1]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, SeparableConv2D
from keras.layers import concatenate, add, Lambda
from keras.layers import LSTM
from keras.models import Model
from keras.regularizers import l2

from keras.applications import imagenet_utils, Xception
from keras.preprocessing.image import img_to_array
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py
import random

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
from helpers import TrainingMonitor
from helpers import Utils

In [3]:
output_path = "../output/"

In [4]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [5]:
import pickle
data = pickle.load(open('../input/datasets/UCF-101-hdf5/full_sequence_rgbmean.pkl', 'rb'))

In [6]:
train_split, val_split = 0.7, 0.15
random.shuffle(data)

train_len = int(train_split * len(data))
train_val_len = int((train_split + val_split) * len(data))
train = data[:train_len]
val = data[train_len:train_val_len]
test = data[train_val_len:]

In [7]:
x_train, y_train = np.array([t["imgs"] for t in train]), [t["label"] for t in train]
x_val, y_val = np.array([t["imgs"] for t in val]), [t["label"] for t in val]
x_test, y_test = np.array([t["imgs"] for t in test]), [t["label"] for t in test]

x_train = pad_sequences(x_train, maxlen=1, truncating="post")
x_val = pad_sequences(x_val, maxlen=1, truncating="post")
x_test = pad_sequences(x_test, maxlen=1, truncating="post")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 60*80*3))
x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 60*80*3))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 60*80*3))

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_val = lb.fit_transform(y_val)
y_test = lb.fit_transform(y_test)

In [8]:
x_val.shape

(30, 1, 14400)

In [9]:
num_classes = y_train.shape[1]
num_classes

8

In [10]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(x_train.shape[1:])))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(256))
model.add(Dense(num_classes)) 
model.add(Activation("sigmoid"))

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
history = model.fit(x_train, y_train,
            batch_size=64,
            epochs=5,
            validation_data=(x_val, y_val),
            shuffle="batch")

Train on 140 samples, validate on 30 samples
Epoch 1/5
140/140 [==============================] - 3s 18ms/step - loss: 0.6902 - acc: 0.6857 - val_loss: 0.6802 - val_acc: 0.8750
Epoch 2/5
140/140 [==============================] - 0s 609us/step - loss: 0.6772 - acc: 0.8750 - val_loss: 0.6637 - val_acc: 0.8750
Epoch 3/5
140/140 [==============================] - 0s 587us/step - loss: 0.6592 - acc: 0.8750 - val_loss: 0.6397 - val_acc: 0.8750
Epoch 4/5
140/140 [==============================] - 0s 594us/step - loss: 0.6331 - acc: 0.8750 - val_loss: 0.6059 - val_acc: 0.8750
Epoch 5/5
140/140 [==============================] - 0s 573us/step - loss: 0.5965 - acc: 0.8750 - val_loss: 0.5604 - val_acc: 0.8750


In [13]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 87.50%; Val: 87.50%; Test: 87.50%


In [15]:
model = Sequential()
model.add(LSTM(256, stateful=True, batch_input_shape=(5, 1, 14400))) 
model.add(Dense(num_classes)) 
model.add(Activation("sigmoid"))

In [16]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
for i in range(5): 
    model.fit(x_train, y_train, epochs=1, batch_size=5)
    model.reset_states()

Epoch 1/1
140/140 [==============================] - 1s 6ms/step - loss: 0.4639 - acc: 0.8286
Epoch 1/1
140/140 [==============================] - 1s 4ms/step - loss: 0.3882 - acc: 0.8750
Epoch 1/1
140/140 [==============================] - 1s 4ms/step - loss: 0.3774 - acc: 0.8750
Epoch 1/1
140/140 [==============================] - 1s 4ms/step - loss: 0.3739 - acc: 0.8750
Epoch 1/1
140/140 [==============================] - 1s 4ms/step - loss: 0.3721 - acc: 0.8750
